# Condo Ownership vs Stock Index Investment

In [5]:
import numpy as np

## Yearly Expenditures for Condo Ownership (3494 hotel-de-ville)

(Property Tax for [Plateau Mont-Royale](http://ville.montreal.qc.ca/pls/portal/docs/PAGE/SERVICE_FIN_EN/MEDIA/DOCUMENTS/2019_PLATEAU_ANG.PDF)) * (Most recent property evaluation for 3494 hotel-de-ville)


In [6]:
plateau_tax_rate = 0.006519 + 0.000025 + 0.001083 + 0.000036 + 0.000591 + 0.000315
municipal_evaluation = 359000
property_tax = plateau_tax_rate * municipal_evaluation

In [7]:
condo_fees = 2400
insurance = 500
maintenance = 300

In [8]:
annual_expense = sum([
    condo_fees,
    insurance,
    maintenance,
    property_tax
])
annual_expense

6276.270999999999

## One-time Expenses for Condo Ownership

In [9]:
welcome_tax = 4500
notary = 1500
title_insurance = 400
inspection = 400

initial_expense = sum([welcome_tax, notary, title_insurance, inspection])
initial_expense

6800

## Mortgage Analysis

In [118]:
interest = 0.03
nper = 25
per = np.arange(nper) + 1
price = 387000
downpayment = 20000

### CMHC Insurance

Interest rates [listed here](https://www.cmhc-schl.gc.ca/en/finance-and-investing/mortgage-loan-insurance/mortgage-loan-insurance-homeownership-programs/cmhc-mortgage-loan-insurance-cost). Mortgage insurance is covered by the lender for loan to value of under 80% (i.e., if your downpayment is at least 20%, this section is of no concern to you).

In [119]:
def mortgage_insurance_rate(p, downpayment):
    ltv = (p - downpayment) / p
    assert ltv > 0
    assert ltv <= 1
    
    if ltv >= 0.95:
        return 0.04
    elif ltv >= 0.90:
        return 0.031
    elif ltv >= 0.85:
        return 0.028
    elif ltv >= 0.80:
        return 0.024
    else:
        return 0
    
mortgage_insurance_rate(price, downpayment)

0.031

In [120]:
price = price + mortgage_insurance_rate(price, downpayment) * price
ppmt = np.ppmt(interest, per, nper, price)
ipmt = np.ipmt(interest, per, nper, price)

In [124]:
fmt = '{0:2d} {4:8.2f} {1:8.2f} {2:8.2f} {3:8.2f}'
p = price
for payment in per:
    index = payment - 1
    p = p + ppmt[index]
    print(fmt.format(payment, ppmt[index], ipmt[index], p, ppmt[index] + ipmt[index]))

cumulative_principal = -np.matmul(ppmt, np.triu(np.ones(nper*nper).reshape((nper, nper))))
# Monthly mortgage payment
yearly_mortgage = - ppmt[0] - ipmt[0]
(ppmt[0] + ipmt[0]) / 12


 1 -22913.55 -10943.64 -11969.91 388053.36
 2 -22913.55 -11271.95 -11641.60 376781.41
 3 -22913.55 -11610.11 -11303.44 365171.31
 4 -22913.55 -11958.41 -10955.14 353212.90
 5 -22913.55 -12317.16 -10596.39 340895.74
 6 -22913.55 -12686.68 -10226.87 328209.06
 7 -22913.55 -13067.28 -9846.27 315141.79
 8 -22913.55 -13459.29 -9454.25 301682.49
 9 -22913.55 -13863.07 -9050.47 287819.42
10 -22913.55 -14278.97 -8634.58 273540.45
11 -22913.55 -14707.33 -8206.21 258833.12
12 -22913.55 -15148.55 -7764.99 243684.56
13 -22913.55 -15603.01 -7310.54 228081.55
14 -22913.55 -16071.10 -6842.45 212010.45
15 -22913.55 -16553.23 -6360.31 195457.21
16 -22913.55 -17049.83 -5863.72 178407.38
17 -22913.55 -17561.33 -5352.22 160846.06
18 -22913.55 -18088.17 -4825.38 142757.89
19 -22913.55 -18630.81 -4282.74 124127.08
20 -22913.55 -19189.74 -3723.81 104937.34
21 -22913.55 -19765.43 -3148.12 85171.91
22 -22913.55 -20358.39 -2555.16 64813.52
23 -22913.55 -20969.14 -1944.41 43844.38
24 -22913.55 -21598.22 -1315.33

-1909.4623550832384

### Comparative Yearly Return

In [88]:
capital = annual_expense + initial_expense + downpayment
total_yearly_cost = annual_expense + yearly_mortgage
appreciation = np.fv(0.02, per, 0, -price) - price
rent = 600 * 12
rental_income = 2100 * 12

yearly_capital = total_yearly_cost - rental_income if rental_income < total_yearly_cost else 0
yearly_income = rental_income - total_yearly_cost if rental_income > total_yearly_cost else 0

(capital, yearly_capital)

(33076.271, 3989.819260998862)

In [81]:
# TODO: add (varying per year) (annual_expense + ipmt[0] - rental_income * 12) compounded by 1.07
# This would be the lost opportunity cost that you'd be otheriwse adding to your stock investments
# (e.g., what is the added capital beyond your initial investment that you must use to sustain your mortgage?)
stocks = np.fv(0.07, per, -yearly_capital, -capital)
stocks

array([ 39381.429231  ,  46127.94853817,  53346.72419684,  61070.81415162,
        69335.59040323,  78178.90099245,  87641.24332292,  97765.94961653,
       108599.38535068, 120191.16158623, 132594.36215826, 145865.78677034,
       160066.21110526, 175260.66514363, 191518.73096468, 208914.86139321,
       227528.72095173, 247445.55067936, 268756.55848791, 291559.33684306,
       315958.30968307, 342065.21062189, 369999.59462642, 399889.38551127,
       431871.46175806])

In [132]:
realestate = (-ppmt + appreciation + rental_income) + (-yearly_capital + ipmt)
realestate = cumulative_principal + appreciation - total_yearly_cost - capital + rental_income
realestate

array([-18142.512     ,   1268.97420883,  21181.40961592,  41608.19478947,
        62563.09975655,  84060.27443573, 106114.25936964, 128739.99676621,
       151952.8418576 , 175768.57458613, 200203.4116266 , 225274.01875496,
       250997.52357339, 277391.52860211, 304474.12474874, 332263.90516617,
       360779.97951024, 390041.98860904, 420070.1195557 , 450885.12123715,
       482508.32031156, 514961.63764758, 548267.60523883, 582449.38360767,
       617530.77971244])

In [134]:
roi = realestate / stocks

rio

In [56]:
first_year = rental_income * 12 - annual_expense - initial_expense + ppmt[0] - ipmt[0]
def net(year):
    if year == 1:
        return first_year
    return net(year - 1) + rental_income * 12 - annual_expense + ppmt[year] - ipmt[year]

In [29]:
np.fv(0.07, per, -80000, -50000)

array([ 133500.        ,  222845.        ,  318444.15      ,
        420735.2405    ,  530186.707335  ,  647299.77684845,
        772610.76122784,  906693.51451379, 1050162.06052976,
       1203673.40476684, 1367930.54310052, 1543685.68111756,
       1731743.67879578, 1932965.73631149, 2148273.33785329,
       2378652.47150302, 2625158.14450824, 2888919.21462381,
       3171143.55964748, 3473123.6088228 , 3796242.2614404 ,
       4141979.21974123, 4511917.76512312, 4907752.00868173,
       5331294.64928946])